In [1]:
import face_recognition
import cv2
import os

# --- FASE 1: CARREGAR TODAS AS PESSOAS CONHECIDAS ---

print("Carregando rostos conhecidos...")

# Listas para guardar os encodings e os nomes
encodings_conhecidos = []
nomes_conhecidos = []

# Caminho para a pasta de pessoas conhecidas
caminho_pasta = "pessoas_conhecidas"

# Percorre todos os arquivos na pasta
for nome_arquivo in os.listdir(caminho_pasta):
    # Pega o nome da pessoa a partir do nome do arquivo (ex: "joao.jpg" -> "joao")
    nome_pessoa = os.path.splitext(nome_arquivo)[0]
    
    try:
        # Carrega a imagem e pega o encoding
        imagem = face_recognition.load_image_file(f"{caminho_pasta}/{nome_arquivo}")
        encoding = face_recognition.face_encodings(imagem)[0]
        
        # Adiciona o encoding e o nome nas listas
        encodings_conhecidos.append(encoding)
        nomes_conhecidos.append(nome_pessoa)
        print(f"  - {nome_pessoa} carregado com sucesso!")
    except IndexError:
        print(f"  - AVISO: Nenhum rosto encontrado em {nome_arquivo}. Pulando.")
    except Exception as e:
        print(f"  - ERRO ao carregar {nome_arquivo}: {e}")

print(f"\nCadastro concluído! {len(nomes_conhecidos)} pessoas registradas.")


# --- FASE 2: CARREGAR A IMAGEM DE TESTE E RECONHECER ---

imagem_teste_path = "fotos_teste/teste_grupo.jpg"
imagem_teste = face_recognition.load_image_file(imagem_teste_path)

# Encontra as localizações e encodings dos rostos na imagem de teste
localizacoes_rostos = face_recognition.face_locations(imagem_teste)
encodings_teste = face_recognition.face_encodings(imagem_teste, localizacoes_rostos)

# Converte a imagem para o formato do OpenCV para desenhar
imagem_para_marcar = cv2.cvtColor(imagem_teste, cv2.COLOR_RGB2BGR)

print(f"\nEncontrados {len(encodings_teste)} rostos na imagem de teste. Iniciando reconhecimento...")

# --- FASE 3: COMPARAR E DESENHAR ---

for (topo, direita, baixo, esquerda), encoding_teste in zip(localizacoes_rostos, encodings_teste):
    # Compara o rosto encontrado com todos os rostos conhecidos
    resultados = face_recognition.compare_faces(encodings_conhecidos, encoding_teste, tolerance=0.6)
    
    nome = "Desconhecido" # Nome padrão se não for reconhecido

    # Se alguma correspondência for encontrada
    if True in resultados:
        # Pega o índice da primeira correspondência
        indice_primeiro_match = resultados.index(True)
        nome = nomes_conhecidos[indice_primeiro_match]

    # Desenha o retângulo e o nome
    if nome == "Desconhecido":
        cor = (0, 0, 255) # Vermelho para desconhecidos
    else:
        cor = (0, 255, 0) # Verde para conhecidos

    cv2.rectangle(imagem_para_marcar, (esquerda, topo), (direita, baixo), cor, 3)
    cv2.putText(imagem_para_marcar, nome, (esquerda + 6, baixo - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, cor, 2)

# Mostra o resultado final
cv2.imshow("Resultado do Reconhecimento Facial", imagem_para_marcar)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("\nProcesso finalizado!")

C:\Users\guilh\AppData\Local\Programs\Python\Python313\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Carregando rostos conhecidos...

Cadastro concluído! 0 pessoas registradas.


FileNotFoundError: [Errno 2] No such file or directory: 'fotos_teste/teste_grupo.jpg'

In [2]:
import face_recognition
import cv2
import os

# --- CORREÇÃO: DEFINIR O DIRETÓRIO DE TRABALHO ---
# Cole aqui o caminho completo que você copiou do explorador de arquivos.
# Exemplo: r"C:\Users\guilh\Desktop\projeto_reconhecimento_facial"
# O 'r' antes da string é importante para evitar problemas com as barras invertidas "\"
caminho_do_projeto = r"C:\Users\guilh\Desktop\projeto_reconhecimento_facial"
os.chdir(caminho_do_projeto)

print(f"Diretório de trabalho alterado para: {os.getcwd()}")
print("-" * 30)


# --- FASE 1: CARREGAR TODAS AS PESSOAS CONHECIDAS ---

print("Carregando rostos conhecidos...")

# Listas para guardar os encodings e os nomes
encodings_conhecidos = []
nomes_conhecidos = []

# Caminho para a pasta de pessoas conhecidas (agora é um caminho relativo)
caminho_pasta = "pessoas_conhecidas"

# Percorre todos os arquivos na pasta
for nome_arquivo in os.listdir(caminho_pasta):
    # Pega o nome da pessoa a partir do nome do arquivo (ex: "joao.jpg" -> "joao")
    nome_pessoa = os.path.splitext(nome_arquivo)[0]
    
    try:
        # Carrega a imagem e pega o encoding
        imagem = face_recognition.load_image_file(f"{caminho_pasta}/{nome_arquivo}")
        encoding = face_recognition.face_encodings(imagem)[0]
        
        # Adiciona o encoding e o nome nas listas
        encodings_conhecidos.append(encoding)
        nomes_conhecidos.append(nome_pessoa)
        print(f"  - {nome_pessoa} carregado com sucesso!")
    except IndexError:
        print(f"  - AVISO: Nenhum rosto encontrado em {nome_arquivo}. Pulando.")
    except Exception as e:
        print(f"  - ERRO ao carregar {nome_arquivo}: {e}")

print(f"\nCadastro concluído! {len(nomes_conhecidos)} pessoas registradas.")


# (COLE ISSO NO LUGAR DO SEU LOOP ATUAL)

# --- NOVA LÓGICA: LOOP PARA PROCESSAR TODAS AS IMAGENS DE TESTE ---

# Define o caminho para a pasta de testes
caminho_pasta_teste = "fotos_teste"
print(f"\nIniciando análise das imagens em '{caminho_pasta_teste}'...\n")

# Loop que passa por CADA ARQUIVO na sua pasta de teste
for nome_arquivo_teste in os.listdir(caminho_pasta_teste):
    # Verifica se o arquivo é realmente uma imagem
    if nome_arquivo_teste.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        imagem_teste_path = os.path.join(caminho_pasta_teste, nome_arquivo_teste)
        print(f"--> Processando imagem: {nome_arquivo_teste}")

        # --- FASE 2: CARREGAR A IMAGEM DE TESTE E RECONHECER ---
        imagem_teste = face_recognition.load_image_file(imagem_teste_path)
        localizacoes_rostos = face_recognition.face_locations(imagem_teste)
        encodings_teste = face_recognition.face_encodings(imagem_teste, localizacoes_rostos)
        imagem_para_marcar = cv2.cvtColor(imagem_teste, cv2.COLOR_RGB2BGR)

        print(f"    Encontrados {len(encodings_teste)} rostos.")

       # Primeiro, adicione esta linha no início do seu script, junto com os outros imports
import numpy as np

# ... (seu código da Fase 1 e 2 continua igual) ...

# --- NOVA LÓGICA: LOOP PARA PROCESSAR TODAS AS IMAGENS DE TESTE ---

# Define o caminho para a pasta de testes
caminho_pasta_teste = "fotos_teste"
print(f"\nIniciando análise das imagens em '{caminho_pasta_teste}'...\n")

# Loop que passa por CADA ARQUIVO na sua pasta de teste
for nome_arquivo_teste in os.listdir(caminho_pasta_teste):
    # Verifica se o arquivo é realmente uma imagem
    if nome_arquivo_teste.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        imagem_teste_path = os.path.join(caminho_pasta_teste, nome_arquivo_teste)
        print(f"--> Processando imagem: {nome_arquivo_teste}")

        # --- FASE 2: CARREGAR A IMAGEM DE TESTE E RECONHECER ---
        imagem_teste = face_recognition.load_image_file(imagem_teste_path)
        localizacoes_rostos = face_recognition.face_locations(imagem_teste)
        encodings_teste = face_recognition.face_encodings(imagem_teste, localizacoes_rostos)
        imagem_para_marcar = cv2.cvtColor(imagem_teste, cv2.COLOR_RGB2BGR)

        print(f"    Encontrados {len(encodings_teste)} rostos.")

        # --- FASE 3: COMPARAR E DESENHAR (VERSÃO MELHORADA) ---
        for (topo, direita, baixo, esquerda), encoding_teste in zip(localizacoes_rostos, encodings_teste):
            nome = "Desconhecido"
            distancias_faciais = face_recognition.face_distance(encodings_conhecidos, encoding_teste)
            
            if len(distancias_faciais) > 0: # Garante que a lista não está vazia
                melhor_indice_match = np.argmin(distancias_faciais)
                menor_distancia = distancias_faciais[melhor_indice_match]
                
                if menor_distancia < 0.56: # Usando uma tolerância mais rigorosa
                    nome = nomes_conhecidos[melhor_indice_match]
            
            cor = (0, 0, 255) if nome == "Desconhecido" else (0, 255, 0)
            cv2.rectangle(imagem_para_marcar, (esquerda, topo), (direita, baixo), cor, 3)
            cv2.putText(imagem_para_marcar, nome, (esquerda + 6, baixo - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, cor, 2)

        # --- BLOCO DE EXIBIÇÃO (NO LUGAR CORRETO AGORA) ---
        nome_da_janela = f"Resultado para: {nome_arquivo_teste}"
        cv2.namedWindow(nome_da_janela, cv2.WINDOW_NORMAL) 
        cv2.imshow(nome_da_janela, imagem_para_marcar)
        print("    Pressione qualquer tecla na janela da imagem para ver a próxima...")
        cv2.waitKey(0) 
        cv2.destroyAllWindows()

print("\nProcesso finalizado! Todas as imagens foram analisadas.")

Diretório de trabalho alterado para: C:\Users\guilh\Desktop\projeto_reconhecimento_facial
------------------------------
Carregando rostos conhecidos...
  - caio.jpg carregado com sucesso!
  - guilherme.jpg carregado com sucesso!
  - joao.jpg carregado com sucesso!
  - pedro.jpg carregado com sucesso!

Cadastro concluído! 4 pessoas registradas.

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
--> Processando imagem: cia.jpg.jpeg
    Encontrados 7 rostos.
--> Processando imagem: festa.jpg.jpeg
    Encontrados 3 rostos.
--> Processando imagem: orbita.jpg.jpeg
    Encontrados 4 rostos.
--> Processando imagem: resenha.jpg.jpeg
    Encontrados 5 rostos.
--> Processando imagem: roca.jpg.jpeg
    Encontrados 3 rostos.

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: cia

In [3]:
import face_recognition
import cv2
import os
import numpy as np

caminho_do_projeto = r"C:\Users\guilh\Desktop\projeto_reconhecimento_facial"
os.chdir(caminho_do_projeto)

print(f"Diretório de trabalho alterado para: {os.getcwd()}")
print("-" * 30)

print("Carregando rostos conhecidos...")

encodings_conhecidos = []
nomes_conhecidos = []

caminho_pasta = "pessoas_conhecidas"

for nome_arquivo in os.listdir(caminho_pasta):
    nome_pessoa = os.path.splitext(nome_arquivo)[0]
    
    try:
        imagem = face_recognition.load_image_file(f"{caminho_pasta}/{nome_arquivo}")
        encoding = face_recognition.face_encodings(imagem)[0]
        
        encodings_conhecidos.append(encoding)
        nomes_conhecidos.append(nome_pessoa)
        print(f"  - {nome_pessoa} carregado com sucesso!")
    except IndexError:
        print(f"  - AVISO: Nenhum rosto encontrado em {nome_arquivo}. Pulando.")
    except Exception as e:
        print(f"  - ERRO ao carregar {nome_arquivo}: {e}")

print(f"\nCadastro concluído! {len(nomes_conhecidos)} pessoas registradas.")

caminho_pasta_teste = "fotos_teste"
print(f"\nIniciando análise das imagens em '{caminho_pasta_teste}'...\n")

for nome_arquivo_teste in os.listdir(caminho_pasta_teste):
    if nome_arquivo_teste.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        imagem_teste_path = os.path.join(caminho_pasta_teste, nome_arquivo_teste)
        print(f"--> Processando imagem: {nome_arquivo_teste}")

        imagem_teste = face_recognition.load_image_file(imagem_teste_path)
        localizacoes_rostos = face_recognition.face_locations(imagem_teste)
        encodings_teste = face_recognition.face_encodings(imagem_teste, localizacoes_rostos)
        imagem_para_marcar = cv2.cvtColor(imagem_teste, cv2.COLOR_RGB2BGR)

        print(f"    Encontrados {len(encodings_teste)} rostos.")

        for (topo, direita, baixo, esquerda), encoding_teste in zip(localizacoes_rostos, encodings_teste):
            nome = "Desconhecido"
            distancias_faciais = face_recognition.face_distance(encodings_conhecidos, encoding_teste)
            
            if len(distancias_faciais) > 0:
                melhor_indice_match = np.argmin(distancias_faciais)
                menor_distancia = distancias_faciais[melhor_indice_match]
                
                if menor_distancia < 0.56:
                    nome = nomes_conhecidos[melhor_indice_match]
            
            cor = (0, 0, 255) if nome == "Desconhecido" else (0, 255, 0)
            cv2.rectangle(imagem_para_marcar, (esquerda, topo), (direita, baixo), cor, 3)
            cv2.putText(imagem_para_marcar, nome, (esquerda + 6, baixo - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, cor, 2)

        nome_da_janela = f"Resultado para: {nome_arquivo_teste}"
        cv2.namedWindow(nome_da_janela, cv2.WINDOW_NORMAL) 
        cv2.imshow(nome_da_janela, imagem_para_marcar)
        print("    Pressione qualquer tecla na janela da imagem para ver a próxima...")
        cv2.waitKey(0) 
        cv2.destroyAllWindows()

print("\nProcesso finalizado! Todas as imagens foram analisadas.")

Diretório de trabalho alterado para: C:\Users\guilh\Desktop\projeto_reconhecimento_facial
------------------------------
Carregando rostos conhecidos...
  - caio.jpg carregado com sucesso!
  - guilherme.jpg carregado com sucesso!
  - joao.jpg carregado com sucesso!
  - pedro.jpg carregado com sucesso!

Cadastro concluído! 4 pessoas registradas.

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: cia.jpg.jpeg
    Encontrados 7 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: festa.jpg.jpeg
    Encontrados 3 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: orbita.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: resenha.jpg.jpeg
    Encontrados 5 r